In [2]:
import torch
import pyro
from pyro.distributions import Poisson

# Simulate some data
true_lambda = 4.0  # our true rate parameter
num_data_points = 1000
data = Poisson(true_lambda).sample((num_data_points,))


In [3]:
def model(data):
    # Prior for lambda. It's generally a good idea to be as uninformative as possible here.
    lambda_ = pyro.sample("lambda", pyro.distributions.Uniform(0, 20))
    
    # Likelihood
    with pyro.plate("data_plate", len(data)):
        pyro.sample("obs", Poisson(lambda_), obs=data)


In [4]:
from pyro.distributions import Normal, constraints

def guide(data):
    # Variational parameters
    lambda_loc = pyro.param("lambda_loc", torch.tensor(5.0))
    lambda_scale = pyro.param("lambda_scale", torch.tensor(1.0), constraint=constraints.positive)

    # Sample lambda
    pyro.sample("lambda", Normal(lambda_loc, lambda_scale))


In [6]:
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam

# Optimizer
optimizer = Adam({"lr": 0.01})

# SVI object
svi = SVI(model, guide, optimizer, loss=Trace_ELBO())

# Training loop
num_steps = 100
for step in range(num_steps):
    loss = svi.step(data)
    if step % 10 == 0:
        print("Step: {} Loss: {}".format(step, loss))

# Get inferred lambda
inferred_lambda = pyro.param("lambda_loc").item()
print(f"Inferred lambda: {inferred_lambda}")


Step: 0 Loss: 2070.1157799959183
Step: 10 Loss: 2070.329275727272
Step: 20 Loss: 2070.175395131111
Step: 30 Loss: 2070.2215341329575
Step: 40 Loss: 2070.892024874687
Step: 50 Loss: 2070.1660608053207
Step: 60 Loss: 2070.2414783239365
Step: 70 Loss: 2070.113800883293
Step: 80 Loss: 2070.2034524679184
Step: 90 Loss: 2070.557561278343
Inferred lambda: 4.019288539886475


In [7]:
import math
import os
import torch
import torch.distributions.constraints as constraints
import pyro
from pyro.optim import Adam
from pyro.infer import SVI, Trace_ELBO
import pyro.distributions as dist

# this is for running the notebook in our testing framework
smoke_test = ('CI' in os.environ)
n_steps = 2 if smoke_test else 2000

assert pyro.__version__.startswith('1.8.6')

# clear the param store in case we're in a REPL
pyro.clear_param_store()

# create some data with 6 observed heads and 4 observed tails
data = []
for _ in range(6):
    data.append(torch.tensor(1.0))
for _ in range(4):
    data.append(torch.tensor(0.0))

def model(data):
    # define the hyperparameters that control the Beta prior
    alpha0 = torch.tensor(10.0)
    beta0 = torch.tensor(10.0)
    # sample f from the Beta prior
    f = pyro.sample("latent_fairness", dist.Beta(alpha0, beta0))
    # loop over the observed data
    for i in range(len(data)):
        # observe datapoint i using the ernoulli likelihood
        pyro.sample("obs_{}".format(i), dist.Bernoulli(f), obs=data[i])

def guide(data):
    # register the two variational parameters with Pyro
    # - both parameters will have initial value 15.0. 
    # - because we invoke constraints.positive, the optimizer 
    # will take gradients on the unconstrained parameters
    # (which are related to the constrained parameters by a log)
    alpha_q = pyro.param("alpha_q", torch.tensor(15.0), 
                         constraint=constraints.positive)
    beta_q = pyro.param("beta_q", torch.tensor(15.0), 
                        constraint=constraints.positive)
    # sample latent_fairness from the distribution Beta(alpha_q, beta_q)
    pyro.sample("latent_fairness", dist.Beta(alpha_q, beta_q))

# setup the optimizer
adam_params = {"lr": 0.0005, "betas": (0.90, 0.999)}
optimizer = Adam(adam_params)

# setup the inference algorithm
svi = SVI(model, guide, optimizer, loss=Trace_ELBO())

# do gradient steps
for step in range(n_steps):
    svi.step(data)
    if step % 100 == 0:
        print('.', end='')

# grab the learned variational parameters
alpha_q = pyro.param("alpha_q").item()
beta_q = pyro.param("beta_q").item()

# here we use some facts about the Beta distribution
# compute the inferred mean of the coin's fairness
inferred_mean = alpha_q / (alpha_q + beta_q)
# compute inferred standard deviation
factor = beta_q / (alpha_q * (1.0 + alpha_q + beta_q))
inferred_std = inferred_mean * math.sqrt(factor)

print("\nBased on the data and our prior belief, the fairness " +
      "of the coin is %.3f +- %.3f" % (inferred_mean, inferred_std))

....................
Based on the data and our prior belief, the fairness of the coin is 0.538 +- 0.090
